In [1]:
from __future__ import annotations

import os
from pathlib import Path
from datetime import datetime

ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks_v2":
    os.chdir(ROOT.parent)
ROOT = Path.cwd().resolve()
assert (ROOT / "src").exists(), f"Not at repo root. CWD={ROOT}"
print("✅ CWD =", ROOT)

def backup_write(rel_path: str, content: str) -> None:
    p = ROOT / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = p.with_suffix(p.suffix + f".bak_{ts}")
        bak.write_text(p.read_text(encoding="utf-8"), encoding="utf-8")
        print("BACKUP:", bak)
    p.write_text(content, encoding="utf-8")
    print("WROTE:", p)

# ---------------- runtime_nodes.py (local-first L0-L5 + persist artifacts) ----------------
backup_write("src/careeragent/langgraph/runtime_nodes.py", r'''
from __future__ import annotations

import asyncio
import concurrent.futures as cf
import json
import math
import os
import re
import urllib.parse
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import httpx


# =========================
# Core helpers
# =========================
def utc_now() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")


def artifacts_root() -> Path:
    return Path("src/careeragent/artifacts").resolve()


def runs_dir(run_id: str) -> Path:
    d = artifacts_root() / "runs" / run_id
    d.mkdir(parents=True, exist_ok=True)
    return d


def daily_dir() -> Path:
    day = datetime.now().strftime("%Y-%m-%d")
    d = artifacts_root() / "daily_jobs" / day
    d.mkdir(parents=True, exist_ok=True)
    return d


def save_json(path: Path, obj: Any) -> str:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2), encoding="utf-8")
    return str(path)


def feed(st: Dict[str, Any], layer: str, agent: str, msg: str) -> None:
    st.setdefault("live_feed", [])
    st["live_feed"].append({"layer": layer, "agent": agent, "message": msg})


def log_attempt(
    st: Dict[str, Any],
    *,
    layer: str,
    agent: str,
    tool: str,
    status: str,
    confidence: float,
    error: Optional[str] = None,
    model: Optional[str] = None,
) -> None:
    st.setdefault("attempts", [])
    st["attempts"].append({
        "layer_id": layer,
        "agent": agent,
        "tool": tool,
        "model": model,
        "status": status,
        "confidence": float(confidence),
        "error": error,
        "at_utc": utc_now(),
    })


def tokenize(text: str) -> List[str]:
    return re.findall(r"[a-zA-Z][a-zA-Z0-9\+\#\.\-]{1,}", (text or "").lower())


def cosine(a: Dict[str, int], b: Dict[str, int]) -> float:
    if not a or not b:
        return 0.0
    dot = sum(v * b.get(k, 0) for k, v in a.items())
    na = math.sqrt(sum(v * v for v in a.values()))
    nb = math.sqrt(sum(v * v for v in b.values()))
    if na == 0 or nb == 0:
        return 0.0
    return float(dot / (na * nb))


def ats_score(resume_text: str) -> float:
    t = (resume_text or "").lower()
    s = 0.0
    if re.search(r"[\w\.-]+@[\w\.-]+\.\w+", t): s += 0.20
    if re.search(r"\+?\d[\d\-\s\(\)]{8,}\d", t): s += 0.10
    for h in ["summary", "skills", "experience", "education", "projects"]:
        if h in t: s += 0.12
    if "-" in resume_text or "•" in resume_text: s += 0.10
    if len(resume_text) > 1200: s += 0.12
    return max(0.0, min(1.0, s))


def interview_chance(skill_overlap: float, exp_align: float, ats: float, market: float) -> float:
    market = max(1.0, float(market))
    score = (0.45 * skill_overlap + 0.35 * exp_align + 0.20 * ats) / market
    return max(0.0, min(1.0, float(score)))


VISA_NEGATIVE = ("unable to sponsor", "cannot sponsor", "no sponsorship", "do not sponsor", "not sponsor", "without sponsorship", "no visa")


COMMON_SKILLS = [
    "python","sql","pandas","numpy","scikit-learn","pytorch","tensorflow","spark","mlflow","dvc",
    "docker","kubernetes","terraform","kafka","airflow","databricks","snowflake",
    "fastapi","streamlit","langchain","langgraph","rag","faiss","chroma","openai","ollama",
    "azure","aws","gcp","sagemaker","ecr","ec2","blob","adls","vector","llm","genai",
]


# =========================
# L0 (always instant)
# =========================
def l0_guard(st: Dict[str, Any]) -> None:
    txt = (st.get("resume_text") or "").lower()
    signals = ["ignore previous instructions", "system prompt", "developer message", "jailbreak", "exfiltrate"]
    blocked = any(s in txt for s in signals)
    log_attempt(st, layer="L0", agent="SanitizeAgent", tool="local.prompt_injection_heuristic",
                status="failed" if blocked else "ok", confidence=0.10 if blocked else 0.95,
                error="prompt_injection" if blocked else None)
    feed(st, "L0", "SanitizeAgent", "Security passed." if not blocked else "Blocked: prompt injection detected.")
    if blocked:
        st["status"] = "blocked"
        st["pending_action"] = "security_blocked"


# =========================
# L2 (local-first parser, no LLM)
# =========================
def l2_parse(st: Dict[str, Any]) -> None:
    resume = st.get("resume_text") or ""
    lines = [x.strip() for x in resume.splitlines() if x.strip()]
    name = lines[0] if lines else "Candidate"

    email = ""
    m = re.search(r"[\w\.-]+@[\w\.-]+\.\w+", resume)
    if m: email = m.group(0)

    phone = ""
    m2 = re.search(r"\+?\d[\d\-\s\(\)]{8,}\d", resume)
    if m2: phone = m2.group(0)

    linkedin = ""
    m3 = re.search(r"https?://(www\.)?linkedin\.com/[^\s]+", resume)
    if m3: linkedin = m3.group(0)

    github = ""
    m4 = re.search(r"https?://(www\.)?github\.com/[^\s]+", resume)
    if m4: github = m4.group(0)

    # Skills: try Skills section first
    skills: List[str] = []
    lower = resume.lower()
    if "skills" in lower:
        # crude section capture
        try:
            sidx = lower.index("skills")
            sub = resume[sidx:sidx+1500]
            tokens = tokenize(sub)
            for sk in COMMON_SKILLS:
                if sk in tokens and sk not in skills:
                    skills.append(sk)
        except Exception:
            pass

    # fallback: scan whole resume for COMMON_SKILLS
    if len(skills) < 8:
        tokens = set(tokenize(resume))
        for sk in COMMON_SKILLS:
            if sk in tokens and sk not in skills:
                skills.append(sk)

    profile = {
        "name": name,
        "contact": {"email": email, "phone": phone, "linkedin": linkedin, "github": github},
        "skills": skills[:40],
        "experience": [],
        "education": [],
    }
    st["profile"] = profile

    # Persist artifact
    rid = str(st.get("run_id") or "run")
    p = runs_dir(rid) / "extracted_profile.json"
    st.setdefault("artifacts", {})
    st["artifacts"]["extracted_profile"] = {"path": save_json(p, profile), "content_type": "application/json"}

    log_attempt(st, layer="L2", agent="ParserAgent", tool="local.resume_parser", status="ok",
                confidence=0.75 if len(skills) >= 8 else 0.55)
    feed(st, "L2", "ParserAgent", f"Profile extracted (skills={len(skills)}).")


# =========================
# L3 (indestructible discovery + persist URLs)
# =========================
async def serper_search(api_key: str, query: str, num: int = 10, tbs: Optional[str] = None, timeout_s: float = 12.0) -> Tuple[bool, List[Dict[str, Any]], Optional[str]]:
    headers = {"X-API-KEY": api_key, "Content-Type": "application/json"}
    payload: Dict[str, Any] = {"q": query, "num": num}
    if tbs: payload["tbs"] = tbs
    try:
        async with httpx.AsyncClient(timeout=timeout_s) as client:
            r = await client.post("https://google.serper.dev/search", headers=headers, json=payload)
        if r.status_code == 403:
            return False, [], "serper_403_quota"
        if r.status_code >= 400:
            return False, [], f"serper_{r.status_code}"
        organic = (r.json().get("organic") or [])
        items = [{"title": x.get("title") or "", "url": x.get("link") or "", "snippet": x.get("snippet") or ""} for x in organic]
        return True, items, None
    except Exception as e:
        return False, [], str(e)


def fallback_board_urls(role: str, location: str) -> List[Dict[str, Any]]:
    q = urllib.parse.quote_plus(f"{role} {location}")
    loc = urllib.parse.quote_plus(location)
    return [
        {"title": f"LinkedIn search: {role}", "url": f"https://www.linkedin.com/jobs/search/?keywords={q}&location={loc}", "snippet": "Fallback job-board search", "source":"fallback"},
        {"title": f"Indeed search: {role}", "url": f"https://www.indeed.com/jobs?q={q}&l={loc}", "snippet": "Fallback job-board search", "source":"fallback"},
        {"title": f"Dice search: {role}", "url": f"https://www.dice.com/jobs?q={q}&location={loc}", "snippet": "Fallback job-board search", "source":"fallback"},
        {"title": f"ZipRecruiter search: {role}", "url": f"https://www.ziprecruiter.com/jobs-search?search={q}&location={loc}", "snippet": "Fallback job-board search", "source":"fallback"},
        {"title": f"Lever search: {role}", "url": f"https://www.google.com/search?q=site:jobs.lever.co+{q}", "snippet": "Fallback via Google", "source":"fallback"},
        {"title": f"Greenhouse search: {role}", "url": f"https://www.google.com/search?q=site:boards.greenhouse.io+{q}", "snippet": "Fallback via Google", "source":"fallback"},
    ]


async def l3_discovery_async(st: Dict[str, Any]) -> None:
    prefs = st.get("preferences") or {}
    roles = prefs.get("target_roles") or [prefs.get("target_role") or "Data Scientist"]
    roles = [str(r).strip() for r in roles if str(r).strip()][:3]
    location = str(prefs.get("location") or "United States")
    recency_h = float(prefs.get("recency_hours") or 36.0)
    tbs = "qdr:d" if recency_h <= 36 else None
    max_jobs = int(prefs.get("max_jobs") or 40)

    api_key = os.getenv("SERPER_API_KEY", "")

    primary = roles[0] if roles else "Data Scientist"
    board_domains = ["linkedin.com/jobs", "indeed.com", "glassdoor.com", "ziprecruiter.com", "monster.com", "dice.com", "jobs.lever.co", "boards.greenhouse.io"]
    queries = [f'site:{d} "{primary}" "{location}"' for d in board_domains]
    queries.extend([f"{r} {location} apply" for r in roles[:2]])
    st["discovery_queries"] = queries

    items: List[Dict[str, Any]] = []

    if api_key:
        async def one(q: str):
            return await serper_search(api_key, q, num=10, tbs=tbs)

        results = await asyncio.gather(*[one(q) for q in queries[:10]], return_exceptions=False)
        for ok, got, err in results:
            log_attempt(st, layer="L3", agent="DiscoveryAgent", tool="serper.search",
                        status="ok" if ok else "failed", confidence=0.75 if (ok and got) else 0.30, error=err)
            if ok:
                for it in got:
                    it["source"] = "serper"
                items.extend(got)

    if not items:
        log_attempt(st, layer="L3", agent="DiscoveryAgent", tool="local.fallback_board_urls", status="ok", confidence=0.55)
        items = fallback_board_urls(primary, location)

    # dedupe + cap
    seen = set()
    jobs_raw: List[Dict[str, Any]] = []
    for it in items:
        url = (it.get("url") or "").strip()
        if not url or url in seen:
            continue
        seen.add(url)
        jobs_raw.append({
            "title": it.get("title") or "",
            "url": url,
            "snippet": it.get("snippet") or "",
            "source": it.get("source") or "unknown",
        })
        if len(jobs_raw) >= max_jobs:
            break

    st["jobs_raw"] = jobs_raw

    rid = str(st.get("run_id") or "run")
    run_path = runs_dir(rid) / "jobs_raw.json"
    day_path = daily_dir() / f"{rid}_jobs_raw.json"

    st.setdefault("artifacts", {})
    st["artifacts"]["jobs_raw"] = {"path": save_json(run_path, jobs_raw), "content_type": "application/json"}
    st["artifacts"]["jobs_raw_daily"] = {"path": save_json(day_path, {"run_id": rid, "at_utc": utc_now(), "jobs": jobs_raw}), "content_type": "application/json"}

    feed(st, "L3", "DiscoveryAgent", f"Discovery completed: {len(jobs_raw)} jobs (saved).")


# =========================
# L4 (scrape + match, timeout-safe)
# =========================
async def fetch_text(url: str, timeout_s: float = 10.0) -> Tuple[bool, str, Optional[str]]:
    try:
        async with httpx.AsyncClient(timeout=timeout_s, follow_redirects=True) as client:
            r = await client.get(url, headers={"User-Agent":"Mozilla/5.0"})
        if r.status_code >= 400:
            return False, "", f"http_{r.status_code}"
        html = r.text
        txt = re.sub(r"<(script|style)[^>]*>.*?</\1>", " ", html, flags=re.S|re.I)
        txt = re.sub(r"<[^>]+>", " ", txt)
        txt = re.sub(r"\s+", " ", txt).strip()
        return True, txt[:12000], None
    except Exception as e:
        return False, "", str(e)


async def l4_match_async(st: Dict[str, Any]) -> None:
    prefs = st.get("preferences") or {}
    max_jobs = int(prefs.get("max_jobs") or 40)
    visa_req = bool(prefs.get("visa_sponsorship_required") or False)

    resume = st.get("resume_text") or ""
    prof = st.get("profile") or {}
    resume_skills = [str(x).lower() for x in (prof.get("skills") or [])]
    ats = ats_score(resume)

    exp_tokens: Dict[str, int] = {}
    for t in tokenize(resume):
        exp_tokens[t] = exp_tokens.get(t, 0) + 1

    jobs = (st.get("jobs_raw") or [])[:max_jobs]
    scored: List[Dict[str, Any]] = []

    async def score_one(j: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        url = j.get("url") or ""
        title = j.get("title") or ""
        snippet = j.get("snippet") or ""
        ok, txt, err = await fetch_text(url, timeout_s=10.0)
        log_attempt(st, layer="L4", agent="ScraperAgent", tool="httpx.get",
                    status="ok" if ok else "failed", confidence=0.65 if ok else 0.25, error=err)

        text = txt if ok and txt else snippet
        low = (text or "").lower()

        if visa_req and any(x in low for x in VISA_NEGATIVE):
            return None

        tokens_job = tokenize(text)
        job_tf: Dict[str, int] = {}
        for t in tokens_job:
            job_tf[t] = job_tf.get(t, 0) + 1

        matched = [s for s in resume_skills if s and s in low]
        skill_overlap = min(1.0, len(set(matched)) / max(1.0, len(set(tokens_job)) / 40.0))
        exp_align = cosine(exp_tokens, job_tf)

        # Market factor: verified data is currently inconclusive → default 1.0
        market = 1.0

        score = interview_chance(skill_overlap, exp_align, ats, market)

        missing = []
        for sk in COMMON_SKILLS:
            if sk in low and sk not in resume_skills:
                missing.append(sk)
        missing = missing[:15]

        return {
            "job_id": url or title,
            "title": title,
            "url": url,
            "source": j.get("source") or "unknown",
            "snippet": snippet,
            "full_text": text,
            "matched_skills": matched[:15],
            "missing_skills": missing,
            "components": {
                "skill_overlap": float(skill_overlap),
                "experience_alignment": float(exp_align),
                "ats_score": float(ats),
                "market_competition_factor": float(market),
            },
            "score": float(score),
            "match_percent": round(float(score) * 100.0, 2),
        }

    # concurrent scoring with cap
    tasks = [score_one(j) for j in jobs]
    results = await asyncio.gather(*tasks, return_exceptions=True)

    for r in results:
        if isinstance(r, dict):
            scored.append(r)

    scored.sort(key=lambda x: float(x.get("score", 0.0)), reverse=True)
    st["jobs_scored"] = scored

    rid = str(st.get("run_id") or "run")
    p = runs_dir(rid) / "jobs_scored.json"
    st.setdefault("artifacts", {})
    st["artifacts"]["jobs_scored"] = {"path": save_json(p, scored), "content_type": "application/json"}

    feed(st, "L4", "MatcherAgent", f"Matched/scored {len(scored)} jobs (saved).")


# =========================
# L5 (rank + HITL)
# =========================
def l5_rank(st: Dict[str, Any]) -> None:
    prefs = st.get("preferences") or {}
    top_n = int(prefs.get("top_n") or 30)
    scored = st.get("jobs_scored") or []
    ranking = scored[:top_n]
    st["ranking"] = ranking

    rid = str(st.get("run_id") or "run")
    p = runs_dir(rid) / "ranking.json"
    st.setdefault("artifacts", {})
    st["artifacts"]["ranking"] = {"path": save_json(p, ranking), "content_type": "application/json"}

    st["status"] = "needs_human_approval"
    st["pending_action"] = "review_ranking"
    feed(st, "L5", "Ranker", f"Ranking ready: {len(ranking)} jobs (saved).")


# =========================
# Public entrypoints used by RunManagerService
# =========================
async def run_single_layer(state: Dict[str, Any], layer: str) -> Dict[str, Any]:
    layer = (layer or "").upper().strip()

    if layer == "L0":
        l0_guard(state)
        return state

    if layer == "L2":
        l2_parse(state)
        return state

    if layer == "L3":
        await l3_discovery_async(state)
        return state

    if layer == "L4":
        await l4_match_async(state)
        return state

    if layer == "L5":
        l5_rank(state)
        return state

    # L6–L9: use your existing nodes_l6_l9.py if present
    if layer in ("L6","L7","L8","L9"):
        from careeragent.langgraph.nodes_l6_l9 import (
            l6_draft_node, l6_evaluator_node,
            l7_apply_node, l7_evaluator_node,
            l8_tracker_node, l8_evaluator_node,
            l9_analytics_node,
        )
        if layer == "L6":
            state.update(await l6_draft_node(state))      # type: ignore[arg-type]
            state.update(await l6_evaluator_node(state))  # type: ignore[arg-type]
            return state
        if layer == "L7":
            state.update(await l7_apply_node(state))      # type: ignore[arg-type]
            state.update(await l7_evaluator_node(state))  # type: ignore[arg-type]
            return state
        if layer == "L8":
            state.update(await l8_tracker_node(state))      # type: ignore[arg-type]
            state.update(await l8_evaluator_node(state))    # type: ignore[arg-type]
            return state
        if layer == "L9":
            state.update(await l9_analytics_node(state))    # type: ignore[arg-type]
            return state

    feed(state, "L1", "Runtime", f"Layer {layer} not implemented.")
    return state


async def approve_ranking_flow(state: Dict[str, Any]) -> Dict[str, Any]:
    from careeragent.langgraph.nodes_l6_l9 import l6_draft_node, l6_evaluator_node
    state["status"] = "running"
    state["pending_action"] = None
    feed(state, "L6", "HITL", "Ranking approved. Generating drafts…")
    state.update(await l6_draft_node(state))      # type: ignore[arg-type]
    state.update(await l6_evaluator_node(state))  # type: ignore[arg-type]
    state["status"] = "needs_human_approval"
    state["pending_action"] = "review_drafts"
    return state


async def approve_drafts_flow(state: Dict[str, Any]) -> Dict[str, Any]:
    from careeragent.langgraph.nodes_l6_l9 import (
        l7_apply_node, l7_evaluator_node,
        l8_tracker_node, l8_evaluator_node,
        l9_analytics_node,
    )
    state["status"] = "running"
    state["pending_action"] = None
    feed(state, "L7", "HITL", "Drafts approved. Applying + tracking + analytics…")
    state.update(await l7_apply_node(state))      # type: ignore[arg-type]
    state.update(await l7_evaluator_node(state))  # type: ignore[arg-type]
    if state.get("status") == "needs_human_approval":
        return state
    state.update(await l8_tracker_node(state))      # type: ignore[arg-type]
    state.update(await l8_evaluator_node(state))    # type: ignore[arg-type]
    if state.get("status") == "needs_human_approval":
        return state
    state.update(await l9_analytics_node(state))    # type: ignore[arg-type]
    state["status"] = "completed"
    state["pending_action"] = None
    feed(state, "L9", "HITL", "Run completed.")
    return state
''')

# ---------------- run_manager_service.py (hard timeout runner + correct step status) ----------------
backup_write("src/careeragent/api/run_manager_service.py", r'''
from __future__ import annotations

import asyncio
import concurrent.futures as cf
import copy
import threading
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, Optional
from uuid import uuid4

from careeragent.services.db_service import SqliteStateStore
from careeragent.langgraph.runtime_nodes import run_single_layer, approve_ranking_flow, approve_drafts_flow


def utc_now() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")


def artifacts_root() -> Path:
    return Path("src/careeragent/artifacts").resolve()


class RunManagerService:
    """
    Description: Indestructible background runner with hard timeouts + state persistence per layer.
    Layer: L8
    """

    def __init__(self) -> None:
        self._store = SqliteStateStore()

    def _runs_dir(self, run_id: str) -> Path:
        d = artifacts_root() / "runs" / run_id
        d.mkdir(parents=True, exist_ok=True)
        return d

    def save_state(self, *, run_id: str, state: Dict[str, Any]) -> None:
        state.setdefault("meta", {})
        state["meta"]["heartbeat_utc"] = utc_now()
        self._store.upsert_state(run_id=run_id, status=str(state.get("status", "unknown")), state=state, updated_at_utc=utc_now())

    def get_state(self, run_id: str) -> Optional[Dict[str, Any]]:
        return self._store.get_state(run_id=run_id)

    def create_run(self, *, resume_filename: str, resume_text: str, resume_bytes: bytes, preferences: Dict[str, Any]) -> Dict[str, Any]:
        run_id = uuid4().hex
        run_dir = self._runs_dir(run_id)
        (run_dir / "resume_upload.bin").write_bytes(resume_bytes)
        (run_dir / "resume_raw.txt").write_text(resume_text, encoding="utf-8")

        thresholds = (preferences.get("thresholds") or {})
        if "default" in thresholds:
            d = float(thresholds["default"])
            thresholds.setdefault("parser", d)
            thresholds.setdefault("discovery", d)
            thresholds.setdefault("match", d)
            thresholds.setdefault("draft", d)

        state: Dict[str, Any] = {
            "run_id": run_id,
            "status": "running",
            "pending_action": None,
            "hitl_reason": None,
            "hitl_payload": {},
            "thresholds": thresholds,
            "max_retries": int(preferences.get("max_refinements", 3)),
            "layer_retry_count": {},
            "preferences": preferences,
            "resume_filename": resume_filename,
            "resume_text": resume_text,
            "profile": {},
            "jobs_raw": [],
            "jobs_scored": [],
            "ranking": [],
            "drafts": {},
            "bridge_docs": {},
            "meta": {
                "created_at_utc": utc_now(),
                "heartbeat_utc": utc_now(),
                "last_layer": None,
                "plan_layers": ["L0","L2","L3","L4","L5"],
            },
            "steps": [],
            "live_feed": [{"layer":"L1","agent":"API","message":"Run created. Starting background pipeline…"}],
            "attempts": [],
            "gates": [],
            "evaluations": [],
            "artifacts": {
                "resume_raw": {"path": str(run_dir / "resume_raw.txt"), "content_type": "text/plain"},
                "resume_upload": {"path": str(run_dir / "resume_upload.bin"), "content_type": "application/octet-stream"},
            },
        }

        self.save_state(run_id=run_id, state=state)
        return state

    def start_background(self, run_id: str) -> None:
        t = threading.Thread(target=self._bg, args=(run_id,), daemon=True)
        t.start()

    def _call_layer(self, state: Dict[str, Any], layer: str) -> Dict[str, Any]:
        st_copy = copy.deepcopy(state)
        return asyncio.run(run_single_layer(st_copy, layer))

    def _bg(self, run_id: str) -> None:
        state = self.get_state(run_id)
        if not state:
            return

        plan = [("L0", 10), ("L2", 20), ("L3", 45), ("L4", 120), ("L5", 15)]

        for layer, tmo in plan:
            if state.get("status") in ("blocked","needs_human_approval","failed","completed"):
                self.save_state(run_id=run_id, state=state)
                return

            state.setdefault("meta", {})
            state["meta"]["last_layer"] = layer
            state.setdefault("steps", []).append({"layer_id": layer, "status": "running", "started_at_utc": utc_now()})
            state.setdefault("live_feed", []).append({"layer": layer, "agent":"Orchestrator", "message": f"Running {layer}…"})
            self.save_state(run_id=run_id, state=state)

            with cf.ThreadPoolExecutor(max_workers=1) as ex:
                fut = ex.submit(self._call_layer, state, layer)
                try:
                    state = fut.result(timeout=tmo)
                except cf.TimeoutError:
                    state["status"] = "needs_human_approval"
                    state["pending_action"] = f"timeout_{layer.lower()}"
                    state.setdefault("live_feed", []).append({"layer": layer, "agent":"TimeoutGuard", "message": f"{layer} timed out after {tmo}s"})
                except Exception as e:
                    state["status"] = "failed"
                    state["pending_action"] = f"error_{layer.lower()}"
                    state.setdefault("live_feed", []).append({"layer": layer, "agent":"CrashGuard", "message": f"{layer} crashed: {e}"})

            # mark step finished (ok/failed/blocked)
            step_status = "ok"
            if str(state.get("pending_action","")).startswith(("timeout_","error_")):
                step_status = "failed"
            if state.get("status") in ("blocked","failed"):
                step_status = "failed"

            state["steps"][-1]["status"] = step_status
            state["steps"][-1]["finished_at_utc"] = utc_now()
            self.save_state(run_id=run_id, state=state)

            if state.get("pending_action") == "review_ranking":
                state["status"] = "needs_human_approval"
                self.save_state(run_id=run_id, state=state)
                return

        if state.get("status") == "running":
            state["status"] = "needs_human_approval"
            state["pending_action"] = "review_ranking"
            state.setdefault("live_feed", []).append({"layer":"L5","agent":"Orchestrator","message":"Ranking ready for review."})
            self.save_state(run_id=run_id, state=state)

    async def handle_action(self, *, run_id: str, action_type: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        state = self.get_state(run_id)
        if not state:
            raise ValueError("run_id not found")

        if action_type == "execute_layer":
            layer = str(payload.get("layer","")).upper()
            state = await asyncio.to_thread(lambda: self._call_layer(state, layer))
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "approve_ranking":
            state = await asyncio.to_thread(lambda: asyncio.run(approve_ranking_flow(copy.deepcopy(state))))
            self.save_state(run_id=run_id, state=state)
            return state

        if action_type == "approve_drafts":
            state = await asyncio.to_thread(lambda: asyncio.run(approve_drafts_flow(copy.deepcopy(state))))
            self.save_state(run_id=run_id, state=state)
            return state

        state.setdefault("live_feed", []).append({"layer":"L5","agent":"HITL","message":f"Unhandled action_type={action_type}"})
        self.save_state(run_id=run_id, state=state)
        return state
''')

print("✅ Patched: runtime_nodes L0–L5 are now local-first + L3/L4 persist job URLs + hard timeouts enabled.")
print("Restart backend and start a NEW run.")

✅ CWD = /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/langgraph/runtime_nodes.py.bak_20260221_181355
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/langgraph/runtime_nodes.py
BACKUP: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/api/run_manager_service.py.bak_20260221_181355
WROTE: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai/src/careeragent/api/run_manager_service.py
✅ Patched: runtime_nodes L0–L5 are now local-first + L3/L4 persist job URLs + hard timeouts enabled.
Restart backend and start a NEW run.


In [ ]:


RUN_ID=7f93696f145648c0a644b36c4752c74e
curl -sS "http://127.0.0.1:8000/status/$RUN_ID" | python -c "import sys,json; s=json.load(sys.stdin); print('steps',[(x.get('layer_id'),x.get('status'),x.get('finished_at_utc')) for x in (s.get('steps') or [])]); print('jobs_raw',len(s.get('jobs_raw') or [])); print('art_jobs_raw',(s.get('artifacts') or {}).get('jobs_raw')); print('art_scored',(s.get('artifacts') or {}).get('jobs_scored')); print('art_rank',(s.get('artifacts') or {}).get('ranking'))"
ls -la "src/careeragent/artifacts/runs/$RUN_ID/" | head

SyntaxError: invalid decimal literal (1313367093.py, line 1)